In [10]:
%load_ext autoreload
%autoreload 2
%cd ../..

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/chenyenpin/Documents/ADAPR5


# Importando bibliotecas

In [33]:
import pandas as pd
import numpy as np
import requests
import os

from dotenv import (find_dotenv,
                    load_dotenv)
from sklearn.model_selection import train_test_split as tests
from sklearn.linear_model import (LinearRegression)

from sklearn.ensemble import (RandomForestRegressor,
                              GradientBoostingRegressor,
                              AdaBoostRegressor,
                              BaggingRegressor,
                              VotingRegressor)
from sklearn import metrics

from sklearn.preprocessing import (MinMaxScaler, 
                                   StandardScaler,
                                   RobustScaler,
                                   PowerTransformer)
import warnings

pd.set_option("display.max_rows",1000)

warnings.filterwarnings("ignore")

In [34]:
def eda(df:pd.DataFrame)->None:
  print("-"*15,"DataFrame shape", "-"*15 )
  print(f"Linhas: {df.shape[0]}, Colunas: {df.shape[1]}")
  print("-"*15,"DataFrame info", "-"*15 )
  print(df.info())
  print("-"*15,"DataFrame isna", "-"*15 )
  print(df.isna().sum())
  print("-"*15,"DataFrame is duplicated", "-"*15 )
  print(df.duplicated().sum())
  print("-"*15,"DataFrame describe numeric", "-"*15 )
  print(df.describe(percentiles=[.01,0.1,.25,.5,.75,.90,.99]))
  print("-"*15,"DataFrame desribe categorical", "-"*15 )
  print(df.describe(exclude=[np.number]))




def date_engineering(df: pd.DataFrame)->pd.DataFrame:
    df["data_inversa"]=pd.to_datetime(df["data_inversa"],format='mixed')
    df["year"]=pd.to_datetime(df["data_inversa"]).dt.year
    df["month"]=pd.to_datetime(df["data_inversa"]).dt.month
    df["quarter"]=pd.to_datetime(df["data_inversa"]).dt.quarter
    df["quarter_end"]=pd.to_datetime(df["data_inversa"]).dt.is_quarter_end
    return df


def date_col(col:pd.Series):
  df["dia_semana"]=df['dia_semana'].str.split("-").str[0]
  return df["dia_semana"]

def time_col(col:pd.Series):
  df["hour"] = df["horario"].str.split(":").str[0]

  return df["hour"]

def convert_lower_case(column):
  '''
  Funcao que retorna a coluna selecionada para letra minuscula
  -param column: coluna da pd.Dataframe
  '''
  return column.lower()



In [13]:
load_dotenv(find_dotenv())
path=os.getenv("path")


In [14]:
df=pd.read_csv(path)
df.head()

,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,10/11/2009,Terça,14:10:00,MG,381.0,495.0,BETIM,Velocidade incompatível,Colisão traseira,Com Vítimas Feridas,...,0,1,0,4,2,NaN,NaN,NaN,NaN,NaN
1,16/10/2009,Sexta,04:35:00,MG,251.0,419.6,GRAO MOGOL,Outras,Colisão frontal,Com Vítimas Fatais,...,5,0,0,5,2,NaN,NaN,NaN,NaN,NaN
2,20/09/2009,Domingo,20:00:00,PI,316.0,24.0,TERESINA,Dormindo,Saída de Pista,Com Vítimas Feridas,...,4,0,0,6,1,NaN,NaN,NaN,NaN,NaN
3,17/05/2009,Domingo,15:45:00,RJ,101.0,429.0,MANGARATIBA,Falta de atenção,Colisão Transversal,Sem Vítimas,...,0,2,0,0,2,NaN,NaN,NaN,NaN,NaN
4,05/07/2009,Domingo,17:30:00,PE,101.0,59.2,RECIFE,Outras,Queda de motocicleta / bicicleta / veículo,Com Vítimas Feridas,...,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN


In [ ]:
# transformando e criando colunas adicionais para data
df=date_engineering(df)
df["hour"]=time_col(df["horario"])

In [ ]:
df=df.drop(columns=["latitude","longitude",
                    "regional","delegacia","uop","ano","municipio"])

In [ ]:
df["date"]=pd.to_datetime(df["data_inversa"])
df["date"].info()

In [ ]:
cat_col=df.select_dtypes(exclude="number").columns.to_list()
num_col=[col for col in df if col not in cat_col]

In [ ]:
num_col=[
    #'br',
 'pessoas',
 #'mortos',
 'feridos_leves',
 'feridos_graves',
 'ilesos',
 'ignorados',
 'feridos',
 'veiculos',
 'year',
 'month',
 'quarter']

In [ ]:
cat_col=[
  #'data_inversa',
 #'dia_semana',
 #'horario',
 'uf',
 #'km',
 #'municipio',
 #'causa_acidente',
 #'tipo_acidente',
 'classificacao_acidente',
 'fase_dia',
 'sentido_via',
 'condicao_metereologica',
 'tipo_pista',
 'tracado_via',
 'uso_solo',
 'hour',
  ]

In [ ]:
for col in cat_col:
  df[col]=df[col].apply(lambda x: convert_lower_case(x))

In [ ]:
df.dropna(inplace=True)

In [ ]:
mapeamento_metereologica = {
    'ceu claro': 'céu claro',
    'céu claro': 'céu claro',  # Tratando as duas formas de 'céu claro' como iguais
    'nublado': 'nublado',
    'sol': 'sol',
    'chuva': 'chuva',
    'garoa/chuvisco': 'chuva',  # Agrupando garoa/chuvisco com chuva
    'vento': 'vento',
    'nevoeiro/neblina': 'nevoeiro',
    'granizo': 'neve',  # Agrupando granizo com neve
    'neve': 'neve',
    'ignorada': 'ignorado',
    'ignorado': 'ignorado'
}

df['condicao_metereologica'] = df['condicao_metereologica'].map(mapeamento_metereologica)

In [ ]:
df['condicao_metereologica'].unique()

In [ ]:
mapeamento_causas = {
    'desobediência à sinalização': 'desobediência',
    'outras': 'outras',
    'dormindo': 'distração',
    'não guardar distância de segurança': 'distância insegura',
    'defeito mecânico em veículo': 'defeito mecânico',
    'falta de atenção': 'distração',
    'animais na pista': 'presença de animais',
    'velocidade incompatível': 'velocidade inadequada',
    'ingestão de álcool': 'álcool/drogas',
    'ultrapassagem indevida': 'ultrapassagem',
    'defeito na via': 'condição da via',
    'condutor dormindo': 'distração',
    'desobediência às normas de trânsito pelo condutor': 'desobediência',
    'falta de atenção à condução': 'distração',
    'pista escorregadia': 'condição da via',
    'falta de atenção do pedestre': 'distração',
    'sinalização da via insuficiente ou inadequada': 'sinalização inadequada',
    'avarias e/ou desgaste excessivo no pneu': 'defeito mecânico',
    'mal súbito': 'mal súbito',
    'defeito mecânico no veículo': 'defeito mecânico',
    'fenômenos da natureza': 'fenômenos naturais',
    'restrição de visibilidade': 'visibilidade reduzida',
    'objeto estático sobre o leito carroçável': 'obstáculo na via',
    'carga excessiva e/ou mal acondicionada': 'carga mal acondicionada',
    'deficiência ou não acionamento do sistema de iluminação/sinalização do veículo': 'falha nos sistemas do veículo',
    'ingestão de substâncias psicoativas': 'álcool/drogas',
    'reação tardia ou ineficiente do condutor': 'falha do condutor',
    'agressão externa': 'agressão',
    'desobediência às normas de trânsito pelo pedestre': 'desobediência',
    'acumulo de areia ou detritos sobre o pavimento': 'condição da via',
    'ingestão de álcool e/ou substâncias psicoativas pelo pedestre': 'álcool/drogas',
    'ingestão de álcool pelo condutor': 'álcool/drogas',
    'curva acentuada': 'condição da via',
    'desrespeitar a preferência no cruzamento': 'desrespeito à preferência',
    'pedestre andava na pista': 'pedestre na via',
    'pista em desnível': 'condição da via',
    'chuva': 'condição climática',
    'demais falhas mecânicas ou elétricas': 'defeito mecânico',
    'pista esburacada': 'condição da via',
    'ausência de reação do condutor': 'falha do condutor',
    'transitar na contramão': 'infração de trânsito',
    'obras na pista': 'condição da via',
    'ingestão de álcool ou de substâncias psicoativas pelo pedestre': 'álcool/drogas',
    'conversão proibida': 'infração de trânsito',
    'trafegar com motocicleta (ou similar) entre as faixas': 'infração de trânsito',
    'condutor deixou de manter distância do veículo da frente': 'distância insegura',
    'iluminação deficiente': 'falha nos sistemas do veículo',
    'acessar a via sem observar a presença dos outros veículos': 'falta de atenção',
    'deficiência do sistema de iluminação/sinalização': 'falha nos sistemas do veículo',
    'fumaça': 'condição da via',
    'manobra de mudança de faixa': 'manobra imprudente',
    'demais fenômenos da natureza': 'fenômenos naturais',
    'falta de acostamento': 'infraestrutura inadequada',
    'acesso irregular': 'infração de trânsito',
    'transitar no acostamento': 'infração de trânsito',
    'condutor usando celular': 'distração',
    'condutor desrespeitou a iluminação vermelha do semáforo': 'infração de trânsito',
    'acumulo de água sobre o pavimento': 'condição da via',
    'afundamento ou ondulação no pavimento': 'condição da via',
    'ausência de sinalização': 'sinalização inadequada',
    'estacionar ou parar em local proibido': 'infração de trânsito',
    'entrada inopinada do pedestre': 'falta de atenção do pedestre',
    'mal súbito do condutor': 'mal súbito',
    'demais falhas na via': 'condição da via',
    'acumulo de óleo sobre o pavimento': 'condição da via',
    'acostamento em desnível': 'infraestrutura inadequada',
    'pedestre cruzava a pista fora da faixa': 'falta de atenção do pedestre',
    'sinalização mal posicionada': 'sinalização inadequada',
    'desvio temporário': 'condição da via',
    'falta de elemento de contenção que evite a saída do leito carroçável': 'infraestrutura inadequada',
    'declive acentuado': 'condição da via',
    'redutor de velocidade em desacordo': 'condição da via',
    'frear bruscamente': 'manobra imprudente',
    'problema com o freio': 'falha nos sistemas do veículo',
    'retorno proibido': 'infração de trânsito',
    'ingestão de substâncias psicoativas pelo condutor': 'álcool/drogas',
    'obstrução na via': 'obstrução na via',
    'área urbana sem a presença de local apropriado para a travessia de pedestres': 'infraestrutura inadequada',
    'problema na suspensão': 'falha nos sistemas do veículo',
    'deixar de acionar o farol da motocicleta (ou similar)': 'falha nos sistemas do veículo',
    'restrição de visibilidade em curvas verticais': 'visibilidade reduzida',
    'restrição de visibilidade em curvas horizontais': 'visibilidade reduzida',
    'semáforo com defeito': 'sinalização inadequada',
    'faixas de trânsito com largura insuficiente': 'infraestrutura inadequada',
    'neblina': 'condição climática',
    'transitar na calçada': 'infração de trânsito',
    'faróis desregulados': 'falha nos sistemas do veículo',
    'sinalização encoberta': 'sinalização inadequada',
    'participar de racha': 'infração de trânsito',
    'modificação proibida': 'infração de trânsito',
    'sistema de drenagem ineficiente': 'infraestrutura inadequada'
}

df['causa_acidente'] = df['causa_acidente'].map(mapeamento_causas)

In [ ]:
mapeamento_acidente = {
    'colisão transversal': 'colisão',
    'saída de pista': 'saída de pista',
    'colisão lateral': 'colisão',
    'colisão com bicicleta': 'colisão',
    'colisão traseira': 'colisão',
    'colisão com objeto fixo': 'colisão',
    'capotamento': 'capotamento',
    'queda de motocicleta / bicicleta / veículo': 'queda',
    'incêndio': 'incêndio',
    'atropelamento de pessoa': 'atropelamento de pessoa',
    'colisão frontal': 'colisão',
    'atropelamento de animal': 'atropelamento de animal',
    'colisão com objeto móvel': 'colisão',
    'tombamento': 'tombamento',
    'derramamento de carga': 'derramamento de carga',
    'danos eventuais': 'outros',
    'colisão com objeto estático': 'colisão',
    'saída de leito carroçável': 'saída de pista',
    'queda de ocupante de veículo': 'queda',
    'atropelamento de pedestre': 'atropelamento de pessoa',
    'engavetamento': 'colisão',
    'colisão com objeto em movimento': 'colisão',
    'colisão com objeto': 'colisão',
    'colisão lateral mesmo sentido': 'colisão',
    'eventos atípicos': 'outros',
    'colisão lateral sentido oposto': 'colisão'
}

df['tipo_acidente'] = df['tipo_acidente'].map(mapeamento_acidente)

In [ ]:
df["dia_semana"]=date_col(df["dia_semana"])
df["dia_semana"].unique()